## Detekcia anomálií a outlierov

Outliery a anomálie je možné hľadať viacerými spôsobmi.

Outliery je jednoduché hľadať napr. pomocou rôznych vizualizácií, ak hľadáme outliery v rámci jednotlivých atribútov alebo v rámci kombinácie hodnôt malého počtu atribútov. 

Z jednoduchých vizualizačných techník pre detekciu outlierov môžeme použiť:

### Úloha 13.7:

Ktoré vizualizačné techniky podporované knižnicou Seaborn môžeme priamo použiť pre detekciu outlierov?
Použite zvolené techniky pre detekciu outlierov v rámci datasetu Titanic. 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
titanic = pd.read_csv("../data/titanic-processed.csv")
titanic.head()

In [ ]:
# YOUR CODE HERE

g = sns.distplot(titanic["fare"])

In [ ]:
# YOUR CODE HERE
g = sns.boxplot(x="pclass", y="fare", data=titanic)

In [ ]:
# YOUR CODE HERE

g = sns.scatterplot(x="pclass", y="fare", data=titanic)

Okrem takýchto vizualizácií je možné outliery detegovať pomocou zhlukovacích algoritmov. V takom prípade je vhodné použiť takú metódu, ktorá nám deteguje málo-početné zhluky, ktoré sú vzdialené od štandardných príkladov. Na detekciu môžeme teda použiť metódy založené na hustote (ako napr. DBSCAN), kde nastavíme faktor vzdialenosti bodov patriacich do zhluku tak, aby sme odlíšili všetky štandardné príklady od tých vzdialených, ktoré považujeme za outliery. 

Na príklade datasetu Titanic si demonštrujeme použitie metódy DBSCAN pre detekciu outlierov z pohľadu atribútov `age` a `fare`. 

In [ ]:
# predspracujeme dáta rovnako ako v píkladoch z predoškých cvičení:
# - odstránime atribúty, ktoré nebudeme používať (napr. duplicitné)
# - binárne a ordinálne atribúty namapujeme na indexy
# - kategorické atribúty bez usporiadania transformujeme pomocou One Hot prístupu

titanic = titanic.drop(columns=['cabin','deck','ticket','title'])
titanic['sex'] = titanic['sex'].map({"male": 0, "female": 1})
titanic['has_family'] = titanic['has_family'].map({False: 0, True: 1})
titanic['fare_ordinal'] = titanic['fare_ordinal'].map({"normal": 0, "more expensive": 1, "most expensive": 2}) 
titanic['age_ordinal'] = titanic['age_ordinal'].map({"child": 0, "young": 1, "adult": 2, "old": 3}) 
titanic = pd.get_dummies(titanic, columns=['embarked', 'title_short'])

Natrénujeme model DBSCAN s definovanou hodnotou parametra `eps`. Skúsime nájsť správnu hodnotu parametra tak, aby vhodne separovala príklady do zhlukov - cieľom je "oddeliť" ouliery od štandardných príkladov. 

Výsledky potom môžeme vykresliť pomocou Seaborn knižnice a jej bodového grafu (scatter plot). 

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=100) # inicializujeme DBSCAN model pre definovanú hodnotu minimálnej vzdialenosti

labels = dbscan.fit_predict(titanic) # natrénujeme model na vstupných dátach

g = sns.scatterplot(x='age', y='fare', hue=labels, data=titanic) # vykreslíme bodový graf, farebne rozlíšený podľa zhlukov

Okrem jednoduchých vizualizačných techník a nízko-rozmerných dát vieme použiť zhlukovanie na detekciu anomálií v dátach. Obvykle je tieto metódy dobré použiť aj tam, kde sa jedná o predikčnú úlohu s veľmi nevybalansovaným cieľovým atribútom - minoritnú triedu tak môžeme "odhaliť" pomocou zhlukovania.

Ako príklad si ukážeme ako detegovať podozrivé transakcie v dátach popisujúcich transakcie vykonané pomocou kreditných kariet. 

Pochopenie a interpretácia dát je obtiažna - jedná sa o transformované príznaky, ktoré sú zakódované, vieme len, že sa jedná o atribúty platiteľa a samotnej platby. 

In [ ]:
from sklearn.preprocessing import normalize # importujeme používané knižnice
from sklearn.metrics import confusion_matrix

data=pd.read_csv("../data/creditcard.csv") # načítame dáta do dátového rámca zo súboru
data.head() # vypíšeme na obrazovku prvých 5 záznamov

Pozrieme sa na distribúciu cieľového atribútu:

In [ ]:
print(data["Class"].value_counts())

In [ ]:
g = sns.countplot(x='Class', data=data)

Dataset transformujeme rovnako ako pri predikčných úlohách - odseparujeme si maticu príznakov a vektor hodnôt cieľového atribútu `Class`. Ten môžeme potom použiť pre verifikáciu. 

In [ ]:
features=data.drop(["Time","Class"],axis=1)
labels=pd.DataFrame(data[["Class"]])

Dátový rámec s maticou príznakov normalizujeme.

In [ ]:
from sklearn.preprocessing import normalize
features=normalize(features)

Teraz skúsime natrénovať zhlukovací model. Cheme natrénovať model tak, aby nám oddelil vhodným spôsobom triedy predstavujúce anomálie od majoritných transakcií. Porovnáme potom výsledky zhlukovania so skutočnými hodnotami, ktoré uchovávame vo vektore hodnôt cieľového atribútu.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

#kmeans=KMeans(n_clusters=2, max_iter=300)
#kmeans.fit(features)
#y_kmeans=kmeans.predict(features)

dbscan = DBSCAN(eps=0.5)
y_dbscan = dbscan.fit_predict(features)

Výsledky zhlukovania môžeme vyjadriť aj početnosťou zhlukov. Tá nám dá aspoň približný odhad kvality zhlukovacieho modelu (pomer príkladov v zhlukoch). Samozjreme, nehovorí ešte nič o tom, či príklady v jednotlivých zhlukoch skutočne zodpovedajú aj priradeniu tried. 

In [ ]:
#clusters, counts = np.unique(y_kmeans, return_counts=True) # pomocou funkcie unique identifikujeme rôzne hodnoty a vrátime aj ich počty
#print(np.asarray((clusters, counts))) 

clusters, counts = np.unique(y_dbscan, return_counts=True) # pomocou funkcie unique identifikujeme rôzne hodnoty a vrátime aj ich počty
print(np.asarray((clusters, counts))) 

In [ ]:
#print(confusion_matrix(labels,y_kmeans))

y_dbscan[y_dbscan == -1] = 1
print(confusion_matrix(labels == 1,y_dbscan == -1))

In [ ]:
Fraud = data[data['Class']==1]
Valid = data[data['Class']==0]

outlier_fraction = len(Fraud)/float(len(Valid))
print(outlier_fraction)

Pre hľadanie outlierov môžeme použiť aj metódu Local Outlier Factor.

In [ ]:
from sklearn.neighbors import LocalOutlierFactor # importujeme knižnice

lof = LocalOutlierFactor(n_neighbors=40, metric='euclidean', contamination = outlier_fraction) # vytvoríme model, počíta sa hustota v okolí každého príkladu (počet susedov)
y_lof = lof.fit_predict(features) # natrénujeme model
#scores_prediction = lof.negative_outlier_factor_ 

In [ ]:
y_lof[y_lof == 1] = 0
y_lof[y_lof == -1] = 1

print(confusion_matrix(labels,y_lof))